<a href="https://colab.research.google.com/github/Sweetmonee/DataAnalytics/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

y_train = pd.read_csv('/kaggle/input/titanic/train.csv')
X_Test = pd.read_csv('/kaggle/input/titanic/test.csv')
gs = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)

    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])


    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("../input/titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# print(X_train,X_test,y_train,y_test)

# 데이터 전처리
X_train1 = pd.get_dummies(X_train[["Pclass", "Sex", "SibSp", "Parch"]])
X_test1 = pd.get_dummies(X_test[["Pclass", "Sex", "SibSp", "Parch"]])
print(X_test1)
y_train1 = y_train['Survived']
y_test1 = y_test['Survived']

# 모델링
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 200, max_depth = 7, random_state = 2023)
rfc.fit(X_train1,y_train1)
rfc_answer = rfc.predict(X_test1)
rfc_proba_answer = rfc.predict_proba(X_test1)[:,1]

#모델 평가
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

rfc_ac_score = accuracy_score(y_test1, rfc_answer)
rfc_auc_score = roc_auc_score(y_test1, rfc_proba_answer)
rfc_f1_score = f1_score(y_test1, rfc_answer)

print(f"랜덤포래스트의 정확도 점수는 {rfc_ac_score}입니다")
print(f'랜덤포레스트의','ROC AUC 값: {0:.4f}'.format(rfc_auc_score))
print(f"랜덤포래스트의 f1 score는 {rfc_f1_score}입니다")

from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt
def roc_curve_plot(y_test1 , rfc_proba_answer):
    # 임곗값에 따른 FPR, TPR 값을 반환.
    fprs , tprs , thresholds = roc_curve(y_test1 ,rfc_proba_answer)

    # ROC Curve를 plot 곡선으로 그림.
    plt.plot(fprs , tprs, label='ROC')
    # 가운데 대각선 점선.
    plt.plot([0, 1], [0, 1], 'k--', label='Random')

    # FPR X 축의 Scale을 0.1 단위로 변경, X,Y 축명 설정등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('FPR( 1 - Sensitivity )')
    plt.ylabel('TPR( Recall )')
    plt.legend()
    plt.show()

roc_curve_plot(y_test1, rfc_proba_answer )